Compare SMCD results from "old" DCP generated by Lan from SMCD vs "local" file generated by running SMCD from the share directory

In [ ]:
import pyhecdss as dss
import pydsm
import pandas as pd

In [ ]:
import pydsm
from pydsm import dcd_calcs
import hvplot.pandas

In [ ]:
info1 = { 'file': 'new/SMCD_Sep2023_1_20230131.dss', 'label': 'new', 'is_monthly': False, 'has_seepage': True}
info2 = { 'file': 'old/SMCD_plan_Sep2019.dss', 'label': 'old', 'is_monthly': True, 'has_seepage': False}
#info1 = { 'file': './SMCD/SMCD/SMCD_pond_Jul2020_1.dss', 'label': 'local', 'is_monthly': False, 'has_seepage': True}
#info2 = { 'file': 'new/SMCD_Sep2023_1_20230131.dss', 'label': 'new', 'is_monthly': False, 'has_seepage': True}


In [ ]:
def get_div_seep_drain_sum(dssfile, is_monthly=False, seepage=True):
    epart = '1MON' if is_monthly else '1DAY'
    result = dcd_calcs.get_dcd_flows(dssfile, epart=epart, seepage=seepage)
    div_sum = dcd_calcs.sum(result[0])
    drain_sum = dcd_calcs.sum(result[1])
    if len(result) > 2:
        seep = result[2]
        seep_sum = dcd_calcs.sum(seep)
    else:
        seep_sum = pd.Series(dtype='float64') 
    if not is_monthly:
        div_sum = div_sum.resample('M').mean()
        drain_sum = drain_sum.resample('M').mean()
        seep_sum = seep_sum.resample('M').mean()

    return div_sum, seep_sum, drain_sum

In [ ]:
ndiv_sum, nseep_sum, ndrain_sum = get_div_seep_drain_sum(info1['file'], info1['is_monthly'], info1['has_seepage'])

In [ ]:
if not nseep_sum.empty:
    ndiv_seep_sum = ndiv_sum + nseep_sum
else:
    ndiv_seep_sum = ndiv_sum

In [ ]:
odiv_sum, oseep_sum, odrain_sum = get_div_seep_drain_sum(info2['file'], info2['is_monthly'], info2['has_seepage'])
if not oseep_sum.empty:
    odiv_seep_sum = odiv_sum + oseep_sum
else:
    odiv_seep_sum = odiv_sum

In [ ]:
div_plot = ndiv_seep_sum.hvplot.step(label=f'{info1["label"]}')*odiv_seep_sum.hvplot.step(label=f'{info2["label"]}')

In [ ]:
drain_plot = ndrain_sum.hvplot.step(label=f'{info1["label"]}')*odrain_sum.hvplot.step(label=f'{info2["label"]}')

In [ ]:
diff_div_seep = ndiv_seep_sum-odiv_seep_sum

diff_drain = ndrain_sum-odrain_sum

diff_plot = diff_div_seep.hvplot.step(label='diff_div_seep')*diff_drain.hvplot.step(label='diff_drain')

In [ ]:
ncd=ndiv_seep_sum-ndrain_sum
ocd=odiv_seep_sum-odrain_sum
cd_plot = ncd.hvplot.step(label=f'{info1["label"]} CD')*ocd.hvplot.step(label=f'{info2["label"]} CD')

In [ ]:
plot_panel = (div_plot.opts(title='Diversion + Seepage')
              +drain_plot.opts(title='Drainage')
              +cd_plot.opts(title='Net Channel Depeltion')).cols(1)

In [ ]:
plot_panel

In [ ]:
if False:
    import holoviews as hv
    hv.extension('bokeh')
    hv.save(plot_panel, 'SMCD-Plot-Div-Drain-Diff.html')